In [0]:
%sql
/*
NoteBook que simula el Job JP_PATRIF_GAR_0300_ARCH_ACEP_DETA_TA Que Genera el Detalle de archivos de Aceptacion del 
SUBPROCESO 364
*/

In [0]:
%run "./startup" 

In [0]:
params = WidgetParams(
    {
        "sr_proceso": str,
        "sr_subproceso": str,
        "sr_subetapa": str,
        "sr_folio": str,
        "sr_instancia_proceso": str,
        "sr_usuario": str,
        "sr_etapa": str,
        "sr_id_snapshot": str,
        "sr_id_archivo": str,
        "sr_fec_arc": str,
        "sr_paso": str,
        "sr_id_archivo_06": str,
        "sr_id_archivo_09": str
    }
)
params.validate()
conf = ConfManager()
query = QueryManager()
db = DBXConnectionManager()
queries_df = query.get_sql_list()
# filtar solo los registros donde la columan 'Archivo SQL' comiencen por el nombre del notebook
display(queries_df.filter(col("Archivo SQL").startswith("GAR")).orderBy("Archivo SQL"))

In [0]:
from datetime import datetime
import pytz

# Define the timezone for Mexico
mexico_tz = pytz.timezone("America/Mexico_City")

# Genera la fecha actual en el formato YYYYMMDD
fecha_actual = datetime.now(mexico_tz).strftime("%Y%m%d")

In [0]:
statement_001 = query.get_statement(
"GAR_0300_ARCH_ACEP_DETA_TA.sql",
SR_FOLIO = params.sr_folio,
SR_SUBPROCESO = params.sr_subproceso,
SR_SYSDATE=fecha_actual
)

In [0]:
if conf.debug:
    df = db.read_data("default", statement_001)  # Ensure all columns in 'statement_001' are explicitly aliased to avoid ambiguity, especially in joins.
if conf.debug:
    display(df)
    display(str(df.count()) + " Registros Totales")

In [0]:
temp_delta = 'DELTA_GAR_0300_ARCH_ACEP_DETA_TA_364' + '_' + params.sr_folio
db.write_delta(temp_delta, db.read_data("default", statement_001), "overwrite")
if conf.debug:
    display(db.read_delta(temp_delta))

In [0]:
from datetime import datetime
import pytz

# Inicializa la clase para subir los archivos
file_manager = FileManager(err_repo_path=conf.err_repo_path)

# Define the timezone for Mexico
mexico_tz = pytz.timezone("America/Mexico_City")

# Genera la fecha actual en el formato TimeStamp
fecha_actual_formato_mex = datetime.now(mexico_tz).strftime("%d-%m-%Y %H:%M:%S.%f")

In [0]:
statement_002 = query.get_statement(
"GAR_0300_ARCH_ACEP_DETA_TA_MERGE.sql",
SR_FOLIO = params.sr_folio,
SR_SUBPROCESO = params.sr_subproceso,
SR_TIMESTAMP=fecha_actual_formato_mex,
SR_USUARIO=params.sr_usuario
)

In [0]:
execution = db.execute_oci_dml(
    statement=statement_002, async_mode=False
)

In [0]:
CleanUpManager.cleanup_notebook(locals())